In [ ]:
from pyspark.sql.functions import col, to_date, when
from pyspark.sql.types import StructType, StructField, StringType, IntegerType, DateType
from pyspark.sql import SparkSession

spark = SparkSession.builder.getOrCreate()

# Read from Bronze tables
students_df = spark.read.table("Bronze.dbo.students")
assignments_df = spark.read.table("Bronze.dbo.assignments")
courses_df = spark.read.table("Bronze.dbo.courses")
submissions_df = spark.read.table("Bronze.dbo.submissions")


StatementMeta(, fa75865e-1943-42eb-8958-4e423157307f, 6, Finished, Available, Finished)

In [ ]:
# Clean and format
students_df_clean = students_df.dropna(subset=["student_id", "name", "email"]) \
    .withColumn("join_date", to_date(col("join_date")))

assignments_df_clean = assignments_df.dropna(subset=["assignment_id", "course_id", "due_date"]) \
    .withColumn("due_date", to_date(col("due_date")))

submissions_df_clean = submissions_df.dropna(subset=["student_id", "assignment_id"]) \
    .withColumn("submitted_at", to_date(col("submitted_at")))

courses_df_clean = courses_df.dropna(subset=["course_id", "course_name"])

StatementMeta(, fa75865e-1943-42eb-8958-4e423157307f, 7, Finished, Available, Finished)

In [ ]:
# ================================
# Silver Table 1: Student + Submission + Assignment
# ================================
student_submission_view = submissions_df_clean.alias("sub") \
    .join(students_df_clean.alias("stu"), col("sub.student_id") == col("stu.student_id")) \
    .join(assignments_df_clean.alias("asg"), col("sub.assignment_id") == col("asg.assignment_id")) \
    .select(
        col("sub.submission_id"),
        col("stu.student_id"),
        col("stu.name").alias("student_name"),
        col("stu.email"),
        col("asg.assignment_id"),
        col("asg.title").alias("assignment_title"),
        col("asg.due_date"),
        col("sub.submitted_at")
    )

StatementMeta(, fa75865e-1943-42eb-8958-4e423157307f, 8, Finished, Available, Finished)

In [ ]:
# Save as Silver table
student_submission_view.write.mode("overwrite").saveAsTable("Silver.dbo.silver_student_submission_view")

StatementMeta(, fa75865e-1943-42eb-8958-4e423157307f, 10, Finished, Available, Finished)

In [ ]:
silver_student_submission_view_df = spark.read.table("Silver.dbo.silver_student_submission_view")
# display(silver_student_submission_view_df)

StatementMeta(, fa75865e-1943-42eb-8958-4e423157307f, 13, Finished, Available, Finished)

SynapseWidget(Synapse.DataFrame, ee5a7b8c-f934-4f74-b101-4216d6f68d60)

In [ ]:
# Silver Table 2: Assignment + Course
# ================================
assignment_course_view = assignments_df_clean.alias("asg") \
    .join(courses_df_clean.alias("crs"), col("asg.course_id") == col("crs.course_id")) \
    .select(
        col("asg.assignment_id"),
        col("asg.title"),
        col("asg.due_date"),
        col("crs.course_id"),
        col("crs.course_name"),
        col("crs.instructor")
    )


StatementMeta(, fa75865e-1943-42eb-8958-4e423157307f, 9, Finished, Available, Finished)

In [ ]:
# Save as Silver table
assignment_course_view.write.mode("overwrite").saveAsTable("Silver.dbo.silver_assignment_course_view")

StatementMeta(, fa75865e-1943-42eb-8958-4e423157307f, 11, Finished, Available, Finished)

In [ ]:
silver_assignment_course_view_df = spark.read.table("Silver.dbo.silver_assignment_course_view")
# display(silver_assignment_course_view_df)

StatementMeta(, fa75865e-1943-42eb-8958-4e423157307f, 14, Finished, Available, Finished)

SynapseWidget(Synapse.DataFrame, 996eaf8c-ddda-4e86-8cd3-be955b6db834)

In [ ]:
# Cross join: All student-assignment combinations
student_assignment_cross = students_df.select("student_id") \
    .crossJoin(assignments_df.select("assignment_id", "due_date"))

# Left join with submissions
submission_status_view = student_assignment_cross.alias("sa") \
    .join(submissions_df_clean.alias("sub"),
          (col("sa.student_id") == col("sub.student_id")) & 
          (col("sa.assignment_id") == col("sub.assignment_id")),
          "left") \
    .withColumn("submitted_at", to_date(col("sub.submitted_at"))) \
    .withColumn("due_date", to_date(col("sa.due_date"))) \
    .withColumn("submission_status",
        when(col("submitted_at").isNull(), "Not submitted")
        .when(col("submitted_at") <= col("due_date"), "Submitted on time")
        .otherwise("Late")
    ) \
    .select(
        col("sa.student_id"),
        col("sa.assignment_id"),
        col("due_date"),
        col("submitted_at"),
        col("submission_status")
    )



StatementMeta(, fa75865e-1943-42eb-8958-4e423157307f, 20, Finished, Available, Finished)

In [ ]:
# Save as Silver table
submission_status_view.write.mode("overwrite").saveAsTable("Silver.dbo.silver_submission_status_view")

StatementMeta(, fa75865e-1943-42eb-8958-4e423157307f, 22, Finished, Available, Finished)

In [ ]:
silver_submission_status_view_df = spark.read.table("Silver.dbo.silver_submission_status_view")
# display(silver_submission_status_view_df)

StatementMeta(, fa75865e-1943-42eb-8958-4e423157307f, 21, Finished, Available, Finished)

SynapseWidget(Synapse.DataFrame, 64b43569-9828-4405-b8c9-f03b0cb056ea)